In [34]:
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (87.0 kB/s)
Reading package li

In [35]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 05:02:03--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.70MB/s    in 0.2s    

2022-11-30 05:02:03 (5.70 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [36]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [37]:
# Read the 2nd CSV file from an S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")


# Show DataFrame
df.show()

#number of rows
df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...|2015-01-31 00:08:00|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...| 

2643619

In [38]:
# Drop null values
dropna_df = df.dropna(how='any')
dropna_df.show()
dropna_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...|2015-01-31 00:08:00|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...| 

2643241

In [39]:
# Print schema
dropna_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [40]:
# Create review dataframe to match review id table
review_df = dropna_df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| REAKC26P07MDN|   28794885|B00Q0K9604|     510387886|2015-01-31 00:08:00|
|R3NU7OMZ4HQIEG|   11488901|B00MBW5O9W|     912374672|2015-01-31 00:08:00|
|R14QJW3XF8QO1P|   43214993|B0084OHUIO|     902215727|2015-01-31 00:08:00|
|R2HB7AX0394ZGY|   12835065|B001GS71K2|     568880110|2015-01-31 00:08:00|
| RGKMPDQGSAHR3|   26334022|B004ABH1LG|     692846826|2015-01-31 00:08:00|
|R1DJCVPQGCV66E|   22283621|B00AX0LFM4|     590674141|2015-01-31 00:08:00|
|R3V52EAWLPBFQG|   14469895|B00DQFZGZ0|     688538603|2015-01-31 00:08:00|
|R3DKO8J1J28QBI|   50896354|B00DIRF9US|     742358789|2015-01-31 00:08:00|
| R764DBXGRNECG|   18440567|B00JRCBFUG|     869798483|2015-01-31 00:08:00|
| RW1853GAT0Z9F|   50502362|B000L3XYZ4|     501118658|2015-01-31 00:08:00|
|R33GITXNUF1AD4|   339301

In [41]:
# Create a dataframe for the product table in our database
product_df = dropna_df.select(["product_id", "product_title"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Q0K9604|(8-Pack) EZwhelp ...|
|B00MBW5O9W|Warren Eckstein's...|
|B0084OHUIO|Tyson's True Chew...|
|B001GS71K2|Soft Side Pet Cra...|
|B004ABH1LG|EliteField 3-Door...|
|B00AX0LFM4|Carlson 68-Inch W...|
|B00DQFZGZ0|Dog Seat Cover Wi...|
|B00DIRF9US|The Bird Catcher ...|
|B00JRCBFUG|Cat Bed - Purrfec...|
|B000L3XYZ4|PetSafe Drinkwell...|
|B00BOEXWFG|Contech ZenDog Ca...|
|B001HBBQKY|Wellness Crunchy ...|
|B007O1FHB0|Rx Vitamins Essen...|
|B001P3NU30|Virbac C.E.T. Enz...|
|B00ZP6HS6S|Kitty Shack - 2 i...|
|B00IP05CUA|Wellness Kittles ...|
|B001U8Y598|OmniPet Anti-Ant ...|
|B011AY4JWO|K9KONNECTION [New...|
|B00DDSHE5A|SUNSEED COMPANY 3...|
|B00PJW5OR8|CXB1983(TM)Cute P...|
+----------+--------------------+
only showing top 20 rows



In [42]:
# Create a dataframe for the customer table in our database
customer_df = dropna_df.groupby("customer_id").agg({"customer_id": "count"})
customer_df.show()


+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   10270641|                 1|
|   18365872|                 1|
|   16711087|                 1|
|   10742726|                 2|
|   41169638|                 1|
|   43622307|                 1|
|   24540309|                 2|
|   28258386|                 1|
|   35329257|                 2|
|   14552054|                 1|
|   14529507|                 5|
|   45392827|                 5|
|   47282953|                 1|
|    8201930|                 1|
|   20109760|                 2|
|   16405801|                 4|
|   15056685|                21|
|   20840575|                 2|
|   39048303|                 1|
|    5596610|                 1|
+-----------+------------------+
only showing top 20 rows



In [43]:
# Create a vine dataframe for the vine table in our database
vine_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| REAKC26P07MDN|          5|            0|          0|   N|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|
|R14QJW3XF8QO1P|          5|            0|          0|   N|
|R2HB7AX0394ZGY|          5|            0|          0|   N|
| RGKMPDQGSAHR3|          5|            0|          0|   N|
|R1DJCVPQGCV66E|          5|            0|          0|   N|
|R3V52EAWLPBFQG|          3|            0|          0|   N|
|R3DKO8J1J28QBI|          2|            0|          0|   N|
| R764DBXGRNECG|          5|            1|          1|   N|
| RW1853GAT0Z9F|          5|            0|          0|   N|
|R33GITXNUF1AD4|          2|            0|          0|   N|
|R1H7AVM81TAYRV|          1|            2|          2|   N|
|R2ZOYAQZNNZZWV|          5|            0|          0|   N|
|R2FN1H3CGW6J8H|          1|            

Postgres Setup

In [44]:
# IMPORTANT: Replace each of these parameters with your own values for your AWS RDS instance
my_aws_endpoint = ' endpoint goes here ' # This is my value; please replace with your own
my_aws_port_number = '5432' # Your value is likely the same, but please double check
my_aws_database_name = 'database1' # This is my value; please replace with your own
my_aws_username = 'postgres' # Your value is likely the same, but please double check
my_aws_password = ' password goes here ' # This is my value; please replace with your own


In [45]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

# Choose to overwrite the existing data. Note that 'append' is probably a smarter choice
# in those situations where the PostgreSQL schema automatically generates the primary key. 
#  But 'overwrite' works best for this little demo. 
mode = 'append' 


In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

review_df.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)